# init

In [ ]:
from huggingface_hub import login

# Replace "YOUR_HF_TOKEN" with your actual token
login()
print("Successfully logged in to Hugging Face!")

Successfully logged in to Hugging Face!


In [2]:
%%capture
import os

!pip install pip3-autoremove
!pip install torch torchvision torchaudio xformers --index-url https://download.pytorch.org/whl/cu128
!pip install unsloth
!pip install transformers==4.55.4
!pip install --no-deps trl==0.22.2

## load model and tokenizer

In [3]:
from unsloth import FastLanguageModel
from transformers import AutoTokenizer
# from vllm import LLM, S20amplingParams

from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-12-17 05:58:07.054185: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765951087.265010      20 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765951087.327685      20 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!


In [4]:
import torch
import gc
if 'model' in globals():
    del model
if 'tokenizer' in globals():
    del tokenizer

# Dọn dẹp rác bộ nhớ
torch.cuda.empty_cache()
gc.collect()
print("🧹 Đã dọn sạch bộ nhớ cũ!")

🧹 Đã dọn sạch bộ nhớ cũ!


In [5]:
max_seq_length = 512
lora_rank = 32 # Larger rank = smarter, but slower
load_in_4bit = True
CHECKPOINT_SOURCE = "/kaggle/input/medical-training/outputs-phase2/checkpoint-500"
custom_model_name ="unsloth/Qwen3-1.7B-unsloth-bnb-4bit"
model, tokenizer = FastLanguageModel.from_pretrained(
model_name = CHECKPOINT_SOURCE,
max_seq_length = max_seq_length,   # Context length - can be longer, but uses more memory
load_in_4bit = True,     # 4bit uses much less memory
# load_in_8bit = False,    # A bit more accurate, uses 2x memory
full_finetuning = False, # We have full finetuning now!
device_map = "cuda:0",
# fast_inference=True,
max_lora_rank = lora_rank,
gpu_memory_utilization = 0.95,

)

==((====))==  Unsloth 2025.12.5: Fast Qwen3 patching. Transformers: 4.55.4.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.41G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

Unsloth 2025.12.5 patched 28 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [6]:
model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ], # Remove QKVO if out of memory
    lora_alpha = lora_rank,
    lora_dropout = 0.05,
    use_gradient_checkpointing = "unsloth", # Enable long context finetuning
    random_state = 3407,

)

Unsloth: Already have LoRA adapters! We shall skip this step.


In [7]:
from datasets import Dataset
import pandas as pd
import os
import torch
from safetensors.torch import load_file
from peft.utils import set_peft_model_state_dict
print(f"💉 Đang nạp kiến thức từ {CHECKPOINT_SOURCE} vào model mới...")

# --- SỬA LỖI TẠI ĐÂY: Dùng load_file của safetensors ---
safetensors_path = os.path.join(CHECKPOINT_SOURCE, "adapter_model.safetensors")

if os.path.exists(safetensors_path):
    adapter_weights = load_file(safetensors_path) # Dùng hàm chuyên dụng
    set_peft_model_state_dict(model, adapter_weights)
    print("✅ Đã nạp kiến thức thành công (Safetensors)!")
else:
    # Fallback nếu file là .bin (PyTorch cũ)
    bin_path = os.path.join(CHECKPOINT_SOURCE, "adapter_model.bin")
    if os.path.exists(bin_path):
        adapter_weights = torch.load(bin_path, map_location="cuda")
        set_peft_model_state_dict(model, adapter_weights)
        print("✅ Đã nạp kiến thức thành công (Bin)!")
    else:
        raise FileNotFoundError(f"❌ Không tìm thấy file adapter_model trong {CHECKPOINT_SOURCE}")

💉 Đang nạp kiến thức từ /kaggle/input/medical-training/outputs-phase2/checkpoint-500 vào model mới...
✅ Đã nạp kiến thức thành công (Safetensors)!


# Load dataset

In [8]:
# @title 3. Format Dữ liệu CHUYÊN BIỆT (CHỈ EN -> VI)
from datasets import Dataset
import pandas as pd
import os

FINAL_DATA_PATH = "/kaggle/input/transformer-medical-dataprocessing/vlsp_medical_cleaned_final.csv"

# 1. Load dữ liệu
try:
    if os.path.exists(FINAL_DATA_PATH):
        df = pd.read_csv(FINAL_DATA_PATH, dtype={'en': str, 'vi': str})
        full_dataset = Dataset.from_pandas(df)
        print(f"✅ Đã load {len(full_dataset)} dòng dữ liệu gốc.")
    else:
        raise FileNotFoundError(f"Không tìm thấy file tại {FINAL_DATA_PATH}")
except Exception as e:
    print(f"❌ Lỗi: {e}")
    raise

# 2. Định nghĩa Prompt (Chỉ cần 1 cái)
sys_prompt_en_to_vi = (
    "Bạn là một biên dịch viên y tế chuyên nghiệp. "
    "Nhiệm vụ của bạn là dịch chính xác văn bản y khoa từ tiếng Anh sang tiếng Việt, "
    "đảm bảo văn phong khoa học và thuật ngữ chính xác."
)

# --- ĐOẠN CẦN SỬA LÀ ĐÂY ---
def formatting_prompts_en_vi_only(examples):
    convos = []
    inputs = examples["en"]
    outputs = examples["vi"]
    
    for en, vi in zip(inputs, outputs):
        # CHỈ GIỮ LẠI CHIỀU EN -> VI
        convos.append([
            {"role": "system", "content": sys_prompt_en_to_vi},
            {"role": "user", "content": str(en)},
            {"role": "assistant", "content": str(vi)},
        ])
        
        # ĐÃ XÓA ĐOẠN VI -> EN Ở ĐÂY
    
    texts = [tokenizer.apply_chat_template(
            c, 
            tokenize=False, 
            add_generation_prompt=False,
            enable_thinking=False 
        ) for c in convos]
    return { "text" : texts }

# --- ÁP DỤNG ---
print("⏳ Đang format dữ liệu chuyên biệt (Chỉ En -> Vi)...")
train_dataset = full_dataset.map(
    formatting_prompts_en_vi_only, # Gọi hàm mới
    batched = True, 
    remove_columns=full_dataset.column_names
)

train_dataset = train_dataset.shuffle(seed=1234)

print("-" * 50)
print(f"✅ Số lượng mẫu huấn luyện (En-Vi Only): {len(train_dataset)}")

✅ Đã load 344426 dòng dữ liệu gốc.
⏳ Đang format dữ liệu chuyên biệt (Chỉ En -> Vi)...


Map:   0%|          | 0/344426 [00:00<?, ? examples/s]

--------------------------------------------------
✅ Số lượng mẫu huấn luyện (En-Vi Only): 344426


In [9]:
# Kiểm tra xem có bao nhiêu tham số đang được "mở khóa" để train
model.print_trainable_parameters()

trainable params: 34,865,152 || all params: 1,755,440,128 || trainable%: 1.9861


In [10]:
from trl import SFTTrainer, SFTConfig
import torch

LAST_CHECKPOINT = "/kaggle/input/medical-training/outputs-phase2/checkpoint-2000"

# Model 1.7B rất nhẹ, bạn có thể tăng Batch Size lên cực cao để chạy nhanh
BATCH_SIZE_PER_GPU = 16
GRAD_ACCUM = 4
training_args = SFTConfig(
    output_dir = "outputs_adapter_en_vi",
    dataset_text_field = "text",
    max_seq_length = max_seq_length, 
    
    per_device_train_batch_size = BATCH_SIZE_PER_GPU, 
    gradient_accumulation_steps = GRAD_ACCUM,
    
    warmup_steps = 100,
    num_train_epochs = 1,
    
    # --- ĐIỀU CHỈNH CỤ THỂ CHO GIỚI HẠN 7 GIỜ ---
    max_steps = 1001,
    # ---

    learning_rate = 1e-5,
    lr_scheduler_type = "constant_with_warmup",
    
    fp16 = True, # Qwen3-1.7B chạy FP16 là ổn định nhất
    bf16 = False,
    optim = "adamw_8bit", 
    weight_decay = 0.01,
    
    # Checkpoint
    logging_steps = 10,
    eval_strategy = "no",
    #eval_steps = 250, # Kiểm tra chất lượng và lưu checkpoint mỗi 250 bước
    save_strategy = "steps",
    save_steps = 500, 
    save_total_limit = 3,
    
    seed = 21,
    report_to = "none",
)

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    #eval_dataset = eval_dataset, 
    args = training_args,
)

print(f"Trainer đang hiểu save_steps là: {trainer.args.save_steps}")
# Nếu nó in ra 250 -> Bạn chưa chạy lại cell cấu hình.
# Nếu nó in ra 500 -> Check lại folder xem có phải file cũ không.
# --- 5. START TRAINING ---
trainer.train()

# --- 6. SAVE FINAL ---
OUTPUT_DIR = "Qwen1.7B_Medical_En_Vi_Specialist" # Tên file đích
model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
print(f"🎉 Hoàn tất! Adapter En-Vi đã lưu tại {OUTPUT_DIR}")

Unsloth: Tokenizing ["text"] (num_proc=8):   0%|          | 0/344426 [00:00<?, ? examples/s]

🦥 Unsloth: Padding-free auto-enabled, enabling faster training.
Trainer đang hiểu save_steps là: 500


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 344,426 | Num Epochs = 1 | Total steps = 1,001
O^O/ \_/ \    Batch size per device = 16 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (16 x 4 x 1) = 64
 "-____-"     Trainable parameters = 34,865,152 of 1,755,440,128 (1.99% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,0.872600
20,0.898200
30,0.881800
40,0.893400
50,0.898900
60,0.895200
70,0.892200
80,0.870900
90,0.888500
100,0.890300


🎉 Hoàn tất! Adapter En-Vi đã lưu tại Qwen1.7B_Medical_En_Vi_Specialist
